## Lab 5 - Create a dynamic viz using Python
This tutorial provides a short demonstration of the folium package, which provides an easy to use interface to Leaflet for Python users. In this tutorial, we are going to use Python and the Python module `Folium` to create first dynamic Geoviz. For this tutorial, you need to install a new Python module `Folium` on your computer. So, open your Anaconda prompt first and then type in `pip install folium`, if that doensn't work, then try `conda install -c conda-forge folium`. 


### Create our first dynamic map
The default tiles are set to `OpenStreetMap`, but `Stamen Terrain`, `Stamen Toner`, `Mapbox Bright`, and `Mapbox Control Room`, and many others tiles are built in. Mapbox needs to be registered. 

In [2]:
!pip install folium

     |████████████████████████████████| 94 kB 2.9 MB/s eta 0:00:01


In [3]:
import folium

# Create a Map instance
m = folium.Map(location=[39.982835, -75.160523], 
               tiles='Stamen Terrain',
               zoom_start=10,
               control_scale=True)

outfp = "firstGeoViz.html"

# save the map as a HTML file
m.save(outfp)

# display the map on your notebook
m

### We can use a list variable to indicate the coordinate of Philadelphia

In [4]:
#Define coordinates of where we want to center our map
phily_coords = [39.982835, -75.160523]

#Create the map
my_map = folium.Map(location = phily_coords, zoom_start = 14)

#Display the map
my_map


### Add a marker of Temple University to the map using default open street map tile
You can add more markers to the map.

In [5]:
#Define the coordinates we want our markers to be at
temple_coords = [39.981183, -75.155338]

#Add markers to the map
folium.Marker(temple_coords, popup = 'Temple University').add_to(my_map)

# If you like, you can add more markers here


#Display the map
my_map

In [8]:
coordinates

### Add line to the map
Add the chestnut hill east line to the map, use `Stamen Terrain` tile

In [6]:
chestnutLine = [('Gravers', 40.077375, -75.201967),
                    ('Chestnut Hill East', 40.080922, -75.207875),
                    ('Wyndmoor Station', 40.073474, -75.196917),
                    ('Mt. Airy', 40.065221, -75.191110),
                    ('Sedgwick', 40.062765, -75.185204),
                    ('Stenton', 40.060498, -75.178856),
                    ('Washington Lane', 40.051092, -75.171567),
                    ('Germantown', 40.037719, -75.171922),
                    ('Wister Avenue', 40.036117, -75.161619),
                    ('Wayne Junction', 40.036117, -75.161619),
                    ('Temple University', 39.981586, -75.149515)]

# get the coordinates for these places
lats = [x[1] for x in chestnutLine]
lons = [x[2] for x in chestnutLine]
coordinates = zip(lats, lons)
type(coordinates)

m = folium.Map(location=[39.963322, -75.161453], 
               zoom_start=11, 
               tiles='Stamen Terrain')

# Create the map with the Chestnut Hill Line
chestnutLineMap = folium.PolyLine(locations=coordinates, weight=5, color = 'red')
m.add_child(chestnutLineMap)


In [7]:
coordinates

### Make choropleth Map using Folium

Reading shapefile using geopandas. We did this many time.

In [8]:
import geopandas as gpd

# read the blood lead data
blood_lead_shp = r'data/child_blood_lead_levels_by_ct.shp'
blood_lead_gdf = gpd.read_file(blood_lead_shp)

# print the first 5 records of the dataframe, used to check the data structure
blood_lead_gdf.head()


,census_tra,data_redac,num_bll_5p,num_screen,perc_5plus,geometry
0,42101000100,0,0.0,100.0,0.0,"POLYGON ((-75.14147 39.95171, -75.14150 39.951..."
1,42101000200,1,NaN,109.0,NaN,"POLYGON ((-75.16238 39.95765, -75.16236 39.957..."
2,42101000300,1,NaN,110.0,NaN,"POLYGON ((-75.17820 39.95981, -75.17743 39.959..."
3,42101000401,1,NaN,61.0,NaN,"POLYGON ((-75.17299 39.95464, -75.17301 39.954..."
4,42101000402,0,0.0,41.0,0.0,"POLYGON ((-75.16333 39.95334, -75.16340 39.953..."


### Here I show you two methods to convert shapefile to geojson file. 
### Option 1. Convert the shapefile into geojson file using ogr

In [9]:
# blood_lead_gdf.to_file("blood_lead.geojson", driver='GeoJson')

import json
import ogr

driver = ogr.GetDriverByName('ESRI Shapefile')
data_source = driver.Open(blood_lead_shp, 0)

fc = {
    'type': 'FeatureCollection',
    'features': []
    }

lyr = data_source.GetLayer(0)
for feature in lyr:
    fc['features'].append(feature.ExportToJson(as_object=True))
    
with open('lead.geojson', 'w') as f:
    json.dump(fc, f)
    

If you find `import gdal` is deprecated, gdal, ogr, osr etc are now part of the osgeo package. Try changing this:

In [10]:
from osgeo import gdal, osr,ogr

driver = ogr.GetDriverByName('ESRI Shapefile')
data_source = driver.Open(blood_lead_shp, 0)

fc = {
    'type': 'FeatureCollection',
    'features': []
    }

lyr = data_source.GetLayer(0)
for feature in lyr:
    fc['features'].append(feature.ExportToJson(as_object=True))
    
with open('lead.geojson', 'w') as f:
    json.dump(fc, f)
    

### Option 2: Convert the shapefile to geojson using Fiona for the conversion
Cite from: https://gis.stackexchange.com/questions/41465/generating-geojson-with-python

In [11]:
import json
import fiona

# create empyt list to store all features of the shapefile
features = []

# open the shapefile using fiona and loop all features in the shapefile
with fiona.collection(blood_lead_shp, "r") as source:
    for feat in source:
        # add the loop feature to the list
        features.append(feat)

# create a dictionary to save the features
my_layer = {
    "type": "FeatureCollection",
    "features": features
}

# write the dictionary of features information to a geojson file
with open("blood_lead.geojson", "w") as featjs:
    featjs.write(json.dumps(my_layer))
    

### Create a choropoleth map based on the generated geojson file

In [12]:
import pandas as pd
import folium

blood_lead_geo = 'blood_lead.geojson'
# lead_df = pd.DataFrame(blood_lead_gdf)

# Create a Map instance
m = folium.Map(location=[39.981183, -75.155338], zoom_start=11, tiles = 'cartodbpositron', control_scale=True)

# Plot a choropleth map
folium.Choropleth(
    geo_data=blood_lead_geo,
    name='choropleth',
    data=blood_lead_gdf,
    columns=['census_tra', 'perc_5plus'],
    key_on='feature.properties.census_tra',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    threshold_scale=[0, 5, 10, 15, 20, 30],
    legend_name='Blood lead level'
).add_to(m)


#Show map
m


## Homework

- 1. Plot your favorate route in Philadelphia on the map using folium. You can find the coordinates from Google Maps
- 2. Add markers at those cuting points of the line
- 3. Create a choropleth maps of blood lead level, using the field of `num_bll_5p`
- 4. Save the result to a `html` and upload both `ipynb` and `html` file to the Canvas

## Reference
1. Quickstart to Folium, https://python-visualization.github.io/folium/quickstart.html
2. Interactive maps on Leaflet, https://automating-gis-processes.github.io/2016/Lesson5-interactive-map-folium.html
